# Paper

> Scripts to facilitate the paper

In [1]:
#| default_exp paper

In [2]:
#| export
#| hide
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from typing import Optional, Any

In [3]:
#| export
def plot_latent_space_with_feature_distributions_paper(latent_representations: np.ndarray,
                                                 labels: np.ndarray,
                                                 features: Optional[np.ndarray] = None,
                                                 feature_names: Optional[list] = None,
                                                 figsize: tuple = (12, 12),  # Adjusted to be a square
                                                 save_path: Optional[str] = None,
                                                 many_classes: bool = False,
                                                 show_legend: bool = True,
                                                 legend_fontsize: int = 12,
                                                 title_fontsize: int = 14,  # Adjusted fontsize for consistent titles
                                                 **kwargs: Any) -> None:
    """
    Plots the latent space with class colors and normalized vertical and horizontal feature distributions in separate subplots.
    """
    # Encode string labels to integers
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    class_names = label_encoder.classes_

    # Use a colormap for better color differentiation
    cmap = plt.get_cmap('tab20')  # You can change 'tab20' to any other colormap
    markers = ['o', 's', '^', 'v', 'D', '<', '>', 'p', '*', 'h', 'H', '8']  # Marker styles

    # Create subplots
    fig = plt.figure(figsize=figsize)
    # Adjust height_ratios and width_ratios to ensure square latent space plot
    gs = fig.add_gridspec(2, 2, height_ratios=[5, 1], width_ratios=[5, 1])

    ax1 = fig.add_subplot(gs[0, 0])  # Latent space plot
    ax2 = fig.add_subplot(gs[1, 0])  # Horizontal distribution plot
    ax3 = fig.add_subplot(gs[0, 1], sharey=ax1)  # Vertical distribution plot

    # Plot the latent representations with class colors
    if many_classes:
        for class_idx, class_name in enumerate(class_names):
            class_mask = (encoded_labels == class_idx)
            color = cmap(class_idx / len(class_names))
            marker = markers[class_idx % len(markers)]
            ax1.scatter(latent_representations[class_mask, 0], latent_representations[class_mask, 1],
                        label=class_name, marker=marker, color=color, s=30, **kwargs)
    else:
        unique_labels = np.unique(encoded_labels)
        for class_idx, class_name in zip(unique_labels, class_names):
            class_mask = (encoded_labels == class_idx)
            color = cmap(class_idx / len(class_names))
            ax1.scatter(latent_representations[class_mask, 0], latent_representations[class_mask, 1],
                        label=class_name, color=color, s=30, **kwargs)

    ax1.set_title('2D Latent Space Visualization', fontsize=title_fontsize)
    ax1.set_xlabel('Dimension 1')
    ax1.set_ylabel('Dimension 2')
    ax1.set_aspect('equal', 'box')  # Ensure the latent space plot is square
    if show_legend:
        ax1.legend(title="Classes", fontsize=legend_fontsize)

    if features is not None and feature_names is not None:
        # Horizontal distribution
        x_min, x_max = np.min(latent_representations[:, 0]), np.max(latent_representations[:, 0])
        x_bins = np.linspace(x_min, x_max, 100)

        for feature_index, feature_name in enumerate(feature_names):
            feature_values = features[:, feature_index]
            avg_feature_values = []

            for i in range(len(x_bins) - 1):
                mask = (latent_representations[:, 0] >= x_bins[i]) & (latent_representations[:, 0] < x_bins[i+1])
                avg_feature_values.append(np.mean(feature_values[mask]) if np.sum(mask) > 0 else 0)

            avg_feature_values = np.array(avg_feature_values)
            # Normalize the feature values
            avg_feature_values = (avg_feature_values - np.min(avg_feature_values)) / (np.max(avg_feature_values) - np.min(avg_feature_values) + 1e-8)
            # Plot the normalized distributions
            ax2.plot((x_bins[:-1] + x_bins[1:]) / 2, avg_feature_values, label=f'{feature_name} distribution')

        ax2.set_title('Normalized Horizontal Feature Distributions', fontsize=title_fontsize)
        ax2.set_xticks([])  # Remove x-axis ticks
        ax2.set_yticks([])  # Remove y-axis ticks
        if show_legend:
            ax2.legend(fontsize=legend_fontsize)

        # Vertical distribution
        y_min, y_max = np.min(latent_representations[:, 1]), np.max(latent_representations[:, 1])
        y_bins = np.linspace(y_min, y_max, 100)

        for feature_index, feature_name in enumerate(feature_names):
            feature_values = features[:, feature_index]
            avg_feature_values = []

            for i in range(len(y_bins) - 1):
                mask = (latent_representations[:, 1] >= y_bins[i]) & (latent_representations[:, 1] < y_bins[i+1])
                avg_feature_values.append(np.mean(feature_values[mask]) if np.sum(mask) > 0 else 0)

            avg_feature_values = np.array(avg_feature_values)
            # Normalize the feature values
            avg_feature_values = (avg_feature_values - np.min(avg_feature_values)) / (np.max(avg_feature_values) - np.min(avg_feature_values) + 1e-8)
            # Plot the normalized distributions
            ax3.plot(avg_feature_values, (y_bins[:-1] + y_bins[1:]) / 2, label=f'{feature_name} distribution')

        # Rotate the title and place it vertically on the left side of the vertical plot
        ax3.set_title('Normalized Vertical Feature Distributions', rotation=90, x=-0.25, y=0.5, fontsize=title_fontsize, verticalalignment='center')
        ax3.set_xticks([])  # Remove x-axis ticks
        ax3.set_yticks([])  # Remove y-axis ticks
        if show_legend:
            ax3.legend(fontsize=legend_fontsize)

    # Remove numbers from latent space plot
    ax1.set_xticks([])  # Remove x-axis ticks
    ax1.set_yticks([])  # Remove y-axis ticks

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Saved plot to {save_path}")
    plt.show()


In [4]:
#| hide
import nbdev; nbdev.nbdev_export()